# [Expressions: Window Functions](https://docs.pola.rs/user-guide/expressions/window-functions/)

In [1]:
import polars as pl

types = (
    "Grass Water Fire Normal Ground Electric Psychic Fighting Bug Steel "
    "Flying Dragon Dark Ghost Poison Rock Ice Fairy".split()
)
type_enum = pl.Enum(types)
# then let's load some csv data with information about pokemon
pokemon = pl.read_csv(
    "https://gist.githubusercontent.com/ritchie46/cac6b337ea52281aa23c049250a4ff03/raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv",
).cast({"Type 1": type_enum, "Type 2": type_enum})
print(pokemon.head())

shape: (5, 13)
┌─────┬───────────────────────┬────────┬────────┬───┬─────────┬───────┬────────────┬───────────┐
│ #   ┆ Name                  ┆ Type 1 ┆ Type 2 ┆ … ┆ Sp. Def ┆ Speed ┆ Generation ┆ Legendary │
│ --- ┆ ---                   ┆ ---    ┆ ---    ┆   ┆ ---     ┆ ---   ┆ ---        ┆ ---       │
│ i64 ┆ str                   ┆ enum   ┆ enum   ┆   ┆ i64     ┆ i64   ┆ i64        ┆ bool      │
╞═════╪═══════════════════════╪════════╪════════╪═══╪═════════╪═══════╪════════════╪═══════════╡
│ 1   ┆ Bulbasaur             ┆ Grass  ┆ Poison ┆ … ┆ 65      ┆ 45    ┆ 1          ┆ false     │
│ 2   ┆ Ivysaur               ┆ Grass  ┆ Poison ┆ … ┆ 80      ┆ 60    ┆ 1          ┆ false     │
│ 3   ┆ Venusaur              ┆ Grass  ┆ Poison ┆ … ┆ 100     ┆ 80    ┆ 1          ┆ false     │
│ 3   ┆ VenusaurMega Venusaur ┆ Grass  ┆ Poison ┆ … ┆ 120     ┆ 80    ┆ 1          ┆ false     │
│ 4   ┆ Charmander            ┆ Fire   ┆ null   ┆ … ┆ 50      ┆ 65    ┆ 1          ┆ false     │
└─────┴────────

## Operations per group

In this example, the group is the `type 1` column. Over specifies what group the operation will be affected by.

In [ ]:
result = pokemon.select(
    pl.col("Name", "Type 1"), # handy way to get a few default columns
    pl.col("Speed").rank("dense", descending=True).over("Type 1").alias("Speed rank"),
)
result

Name,Type 1,Speed rank
str,enum,u32
"""Bulbasaur""","""Grass""",6
"""Ivysaur""","""Grass""",3
"""Venusaur""","""Grass""",1
"""VenusaurMega Venusaur""","""Grass""",1
"""Charmander""","""Fire""",7
…,…,…
"""Moltres""","""Fire""",5
"""Dratini""","""Dragon""",3
"""Dragonair""","""Dragon""",2


In [3]:
pokemon.select(
    pl.col("Name", "Type 1", "Type 2"),
    pl.col("Speed")
    .rank("dense", descending=True)
    .over("Type 1", "Type 2")
    .alias("Speed rank")
)

Name,Type 1,Type 2,Speed rank
str,enum,enum,u32
"""Bulbasaur""","""Grass""","""Poison""",6
"""Ivysaur""","""Grass""","""Poison""",3
"""Venusaur""","""Grass""","""Poison""",1
"""VenusaurMega Venusaur""","""Grass""","""Poison""",1
"""Charmander""","""Fire""",null,7
…,…,…,…
"""Moltres""","""Fire""","""Flying""",2
"""Dratini""","""Dragon""",null,2
"""Dragonair""","""Dragon""",null,1


In [7]:
result = (
    pokemon.group_by("Type 1")
    .agg(
        pl.col("Name"),
        pl.col("Speed").rank("dense", descending=True).alias("Speed rank"),
    )
    .select(pl.col("Name"), pl.col("Type 1"), pl.col("Speed rank"))
    .explode("Name", "Speed rank")
)

print(result)

shape: (163, 3)
┌───────────────────────────┬────────┬────────────┐
│ Name                      ┆ Type 1 ┆ Speed rank │
│ ---                       ┆ ---    ┆ ---        │
│ str                       ┆ enum   ┆ u32        │
╞═══════════════════════════╪════════╪════════════╡
│ Charmander                ┆ Fire   ┆ 7          │
│ Charmeleon                ┆ Fire   ┆ 6          │
│ Charizard                 ┆ Fire   ┆ 2          │
│ CharizardMega Charizard X ┆ Fire   ┆ 2          │
│ CharizardMega Charizard Y ┆ Fire   ┆ 2          │
│ …                         ┆ …      ┆ …          │
│ Tauros                    ┆ Normal ┆ 3          │
│ Ditto                     ┆ Normal ┆ 16         │
│ Eevee                     ┆ Normal ┆ 14         │
│ Porygon                   ┆ Normal ┆ 18         │
│ Snorlax                   ┆ Normal ┆ 19         │
└───────────────────────────┴────────┴────────────┘


## Mapping results to dataframe rows

In [10]:
result = athletes.select(
    pl.col("athlete", "rank").sort_by(pl.col("rank")).over(pl.col("country")),
    pl.col("country"),
)

print(result)

NameError: name 'athletes' is not defined

does npt work, because we are missing athletes

In [11]:
result = athletes.with_columns(
    pl.col("rank").sort().over(pl.col("country"), mapping_strategy="join"),
)

print(result)

NameError: name 'athletes' is not defined

### Windowed aggregation expressions

In [12]:
pokemon.select(
    pl.col("Name", "Type 1", "Speed"),
    pl.col("Speed").mean().over(pl.col("Type 1")).alias("Mean speed in group"),
)

Name,Type 1,Speed,Mean speed in group
str,enum,i64,f64
"""Bulbasaur""","""Grass""",45,54.230769
"""Ivysaur""","""Grass""",60,54.230769
"""Venusaur""","""Grass""",80,54.230769
"""VenusaurMega Venusaur""","""Grass""",80,54.230769
"""Charmander""","""Fire""",65,86.285714
…,…,…,…
"""Moltres""","""Fire""",90,86.285714
"""Dratini""","""Dragon""",50,66.666667
"""Dragonair""","""Dragon""",70,66.666667


## More examples

In [30]:
pokemon.sort("Type 1").select(
    pl.col("Type 1").head(3).over("Type 1", mapping_strategy='explode'),
    pl.col("Name")
    .sort_by(pl.col("Speed"), descending=True)
    .head(3)
    .over("Type 1", mapping_strategy='explode')
    .alias("fastest / group"),
    pl.col("Name")
    .sort_by(pl.col("Attack"), descending=True)
    .head(3)
    .over("Type 1", mapping_strategy='explode')
    .alias("strongest / group"),
    pl.col("Name")
    .sort()
    .head(3)
    .over("Type 1", mapping_strategy='explode')
    .alias("sorted_by_alfabet")
)

Type 1,fastest / group,strongest / group,sorted_by_alfabet
enum,str,str,str
"""Grass""","""Venusaur""","""Victreebel""","""Bellsprout"""
"""Grass""","""VenusaurMega Venusaur""","""VenusaurMega Venusaur""","""Bulbasaur"""
"""Grass""","""Victreebel""","""Exeggutor""","""Exeggcute"""
"""Water""","""Starmie""","""GyaradosMega Gyarados""","""Blastoise"""
"""Water""","""Tentacruel""","""Kingler""","""BlastoiseMega Blastoise"""
…,…,…,…
"""Rock""","""Kabutops""","""Kabutops""","""Geodude"""
"""Ice""","""Jynx""","""Articuno""","""Articuno"""
"""Ice""","""Articuno""","""Jynx""","""Jynx"""


This was a difficult chapter, maybe i need to redo this one.